#CIFAR-100で画像分類を行ってみた

##ライブラリのインポート

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import json
from PIL import Image
import torch
import torchvision
from torchvision import models,transforms 
%matplotlib inline

In [2]:
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
from torchvision import models,transforms 

##データの読み込み〜DataLoaderの作成

前処理クラスの作成

In [3]:
#transformの作成
#なんかtrainとvalidation機能してなくないか…？
class CIFARTransform():
  def __init__(self,resize,mean,std):
    self.transform = {
        "train":transforms.Compose([
             transforms.Resize(resize),
             transforms.RandomHorizontalFlip(),
             transforms.ToTensor(),
             transforms.Normalize(mean,std),
        ]),
        "val":transforms.Compose([
             transforms.Resize(resize),
             transforms.ToTensor(),
             transforms.Normalize(mean,std)                    
        ])
    }

  def __call__(self,img,phase="train"):
    return self.transform[phase](img)  

In [4]:
resize = 32
mean = (0.485,0.456,0.406)
std = (0.229,0.224,0.225)

transform = CIFARTransform(resize,mean,std)

データセットをロード

In [5]:
#CIFAR-100データセットの読み込み
trainandvalset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=True,
                                        download=True,
                                        transform=transform)
     
testset = torchvision.datasets.CIFAR10(root='./data', 
                                        train=False, 
                                        download=True, 
                                        transform=transform)




  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


サブセットの作成

In [7]:
def make_subset_list(dataset,label_list):
  subset_list = []
  for i in range(len(dataset)):
    img_transformed,label = dataset[i][0],dataset[i][1]
    if label in label_list:
      label_mod = label_list.index(label)
      subset_list.append((img_transformed,label_mod))
  return subset_list    

In [8]:
tsglive_label = [4,30,55,72,95,32,67,73,91,6,7,14,3,42,43,88,97,15,19,21,31,38,34,63,64,66,75,26,45,77,79,27,44,78,93,36,50,65,74,80]
len(tsglive_label)

40

In [27]:
trainset,valset = data.random_split(trainandvalset,[40000,10000])
train_subset_list = make_subset_list(trainset,tsglive_label)
val_subset_list = make_subset_list(valset,tsglive_label)
test_subset_list = make_subset_list(testset,tsglive_label)

In [28]:
train_subset_list[2]

(tensor([[[-1.2959, -1.2617, -1.2274,  ..., -1.4158, -1.4843, -1.5528],
          [-1.2959, -1.2788, -1.2274,  ..., -1.4329, -1.4843, -1.5357],
          [-1.2274, -1.2445, -1.2617,  ..., -1.4500, -1.4843, -1.5357],
          ...,
          [-1.5014, -1.5528, -1.4329,  ..., -0.3883, -1.3130, -1.4158],
          [-1.6213, -1.6042, -1.4672,  ...,  0.0398, -0.8335, -1.2103],
          [-1.7583, -1.7412, -1.6213,  ...,  0.1083, -0.5082, -1.0390]],
 
         [[-1.1779, -1.1429, -1.1078,  ..., -1.2129, -1.2829, -1.3529],
          [-1.1604, -1.1604, -1.1078,  ..., -1.2304, -1.2829, -1.3354],
          [-1.0903, -1.1253, -1.1429,  ..., -1.2479, -1.2829, -1.3354],
          ...,
          [-1.2479, -1.1604, -1.0553,  ...,  0.3102, -0.8102, -1.0728],
          [-1.3704, -1.2129, -1.0903,  ...,  0.7829, -0.2850, -0.8102],
          [-1.5105, -1.3529, -1.2479,  ...,  0.8704,  0.0651, -0.6176]],
 
         [[-0.8981, -0.8807, -0.8458,  ..., -0.9853, -1.0550, -1.1247],
          [-0.8633, -0.8807,

In [11]:
#この中で、指定40クラス以外のデータを除いたサブセットを作る

class TsgLiveSubset(data.Dataset):
  """
  dataset: 元々のデータセット（torchvision.datasets)
  class_list: クラスラベルのインデックスが格納されたリスト

  """

  def __init__(self, subset_list, label_list):
      self.subset_list = subset_list
      self.label_list = label_list
      

  def __getitem__(self, idx):
      #class_listに格納されたラベルを正解とするデータのみを取り出す
      self.num = 0
      img_transformed,label = self.subset_list[idx][0],self.subset_list[idx][1]
      return img_transformed,label

  def __len__(self):
    return len(self.subset_list)


In [29]:
trainset_tsglive = TsgLiveSubset(train_subset_list,tsglive_label)
valset_tsglive = TsgLiveSubset(val_subset_list,tsglive_label)
testset_tsglive = TsgLiveSubset(test_subset_list,tsglive_label)

print(len(trainset_tsglive))

16006


データローダーの作成

In [30]:
batch_size = 32
trainloader_teglive = data.DataLoader(trainset_tsglive,batch_size,shuffle=True)
valloader_tsglive = data.DataLoader(valset_tsglive,batch_size,shuffle=False)
testloader_tsglive = data.DataLoader(testset_tsglive,batch_size,shuffle=False)

In [31]:
dataloaders_dict = {"train":trainloader_teglive,"val":valloader_tsglive}

##モデル・損失関数・最適化手法の選択

モデル選択

In [15]:
#モデル選択
#なんかEfficientNetってやつがいいらしいと聞いて
#スペック・スキル的に学習を全部するのはきついので学習済みパラメータを導入
!pip install efficientnet-pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=25e44c25f80b9de303d1b4771b4d7596530ea0fe1d1f5e0b6ec4b15ce7f121f2
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [16]:
from efficientnet_pytorch import EfficientNet

model = EfficientNet. from_pretrained("efficientnet-b7")

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7


In [ ]:
model

In [18]:
#40クラス用に改変
model._fc = nn.Linear(in_features=2560,out_features=40)

損失関数の定義


In [19]:
criterion = nn.CrossEntropyLoss()

最適化手法の定義

In [21]:
#転移学習部分を抽出
params_to_update = []
params_update_names = ["_fc.weight","_fc.bias"]
for name,param in model.named_parameters():
  if name in params_update_names:
    param.requires_grad = True
    params_to_update.append(param)

  else:
    param.requires_grad = False

In [22]:
params_to_update[1]

Parameter containing:
tensor([ 0.0099,  0.0053,  0.0102,  0.0017, -0.0137,  0.0186, -0.0093,  0.0043,
        -0.0137, -0.0070, -0.0046, -0.0049, -0.0068, -0.0072, -0.0045,  0.0070,
        -0.0040, -0.0012, -0.0011,  0.0172,  0.0018,  0.0141,  0.0119,  0.0191,
         0.0104, -0.0025, -0.0196,  0.0057,  0.0002,  0.0165,  0.0031, -0.0147,
         0.0038, -0.0149, -0.0196, -0.0009,  0.0075,  0.0059, -0.0082,  0.0041],
       requires_grad=True)

In [23]:
optimizer = optim.Adam(params=params_to_update,lr= 0.0001)

##学習・モデルの評価

In [24]:
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
  for epoch in range(num_epochs):
    print("Epoch:{}/{}".format(epoch+1,num_epochs))
    print("======================================")

    for phase in ["train","val"]:
      if phase == "train":
        net.train() #訓練モードに
      else:
        net.eval() #検証モードに

      epoch_loss = 0.0 #epochの損失和
      epoch_corrects = 0 #epochの正解数

      #未学習時の検証性能を確かめるため、epoch=0の訓練は省略
      if (epoch == 0) and (phase == 'train'): 
        continue
      #データローダからミニバッチを取り出すループ
      for inputs,labels in tqdm(dataloaders_dict[phase]):
        #勾配初期化
        optimizer.zero_grad()

        #順伝播(forward)計算
        with torch.set_grad_enabled(phase=='train'):
          outputs = net(inputs) #model(入力)で順伝播してくれるやつ
          loss = criterion(outputs,labels) #nn.criterion(出力、正解ラベル)
          _,preds = torch.max(outputs,1) 

          #訓練時のみ逆伝播
          if phase == 'train':
            loss.backward()
            optimizer.step() #optimizer.stepでパラメータを更新する


          #iteration結果の計算
          #loss合計を更新（なんでepoch全体で足してるんだ？？）
          epoch_loss += loss.item()*inputs.size(0)
          """
          ここでなにしてるのかと言いますと
          モチベーションとしては
          "epochごとのlossの総和を取りたい！！"
          そのために、各iterationごとに
          "バッチごとのlossの平均"* "バッチ数"をたしあわせている
          """
          #正解数の合計を更新
          epoch_corrects += torch.sum(preds==labels.data)

      #epochごとのlossと正解率を表示
      epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset) #損失平均
      #1epochは、バッチ抽出を繰り返してのべdataset全体と同じ枚数を抽出した時の回数だから
      epoch_acc = epoch_corrects.double()/len(dataloaders_dict[phase].dataset)

      print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))


In [33]:
#実行！！！
num_epochs = 20
train_model(model,dataloaders_dict,criterion,optimizer,num_epochs)

Epoch:1/20


100%|██████████| 125/125 [01:44<00:00,  1.20it/s]


val Loss: 3.1128 Acc: 0.4089
Epoch:2/20


100%|██████████| 501/501 [09:20<00:00,  1.12s/it]


train Loss: 2.9491 Acc: 0.3990


...

val Loss: 1.8180 Acc: 0.4875
Epoch:20/20


100%|██████████| 501/501 [09:13<00:00,  1.10s/it]


train Loss: 1.8189 Acc: 0.4684


100%|██████████| 125/125 [01:46<00:00,  1.18it/s]

val Loss: 1.7718 Acc: 0.4855
